**Installling Dependencies**


---


numpy 1.16.1 is needed to work with dataset

In [ ]:
!pip install numpy==1.16.1

In [11]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D

**Importing Data**

In [2]:
data = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = data.load_data(num_words = 10000)
print(test_data[0])

17464789/17464789 [==============================] - 0s 0us/step
[1, 591, 202, 14, 31, 6, 717, 10, 10, 2, 2, 5, 4, 360, 7, 4, 177, 5760, 394, 354, 4, 123, 9, 1035, 1035, 1035, 10, 10, 13, 92, 124, 89, 488, 7944, 100, 28, 1668, 14, 31, 23, 27, 7479, 29, 220, 468, 8, 124, 14, 286, 170, 8, 157, 46, 5, 27, 239, 16, 179, 2, 38, 32, 25, 7944, 451, 202, 14, 6, 717]


**Index Mapping**

In [3]:
word_index = keras.datasets.imdb.get_word_index()
word_index = {k:(v+3) for k, v in word_index.items()}

word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

1641221/1641221 [==============================] - 0s 0us/step


In [4]:
def decode_review(text):
  return " ".join([reverse_word_index.get(i, "?") for i in text])

In [5]:
print(decode_review(test_data[0]))

<START> please give this one a miss br br <UNK> <UNK> and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite <UNK> so all you madison fans give this a miss


**Pre-Processing**

In [6]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=word_index["<PAD>"], padding="post", maxlen=250)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=word_index["<PAD>"], padding="post", maxlen=250)

In [7]:
x_val = train_data[:10000]
x_train = train_data[10000:]

y_val = train_labels[:10000]
y_train = train_labels[10000:]

**Building The Model**

In [22]:
model = keras.Sequential()
model.add(keras.layers.Embedding(10000, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation="ReLU"))
model.add(keras.layers.Dense(1, activation="sigmoid"))

model.summary()

model.compile(optimizer="adam", loss=binary_crossentropy, metrics=["accuracy"])

# fitModel = model.fit(x_train, y_train, epochs=40, batch_size=512, validataion_data=(x_val, y_val), verbose=1)
fitModel = model.fit(x_train, y_train, epochs=40, batch_size=512, validation_data=(x_val, y_val), verbose=1)

results = model.evaluate(test_data, test_labels)

print(results)

test_review = test_data[0]
test_review = np.expand_dims(test_review, axis=0) # Add a batch dimension
predict = model.predict(test_review)
print("Review: ")
print(decode_review(test_review[0]))
print("Prediction: " +str(predict[0]))
print("Actual: " + str(test_labels[0]))
print(results)

model.save("model.h5")


# CODE ABOVE CAN BE WRITTEN AS:
# model = Sequential([
#     Embedding(10000, 16),
#     GlobalAveragePooling1D(),
#     Dense(16, activation='relu'),
#     Dense(1, activation='sigmoid')
# ])

# model.compile(loss=binary_crossentropy, optimizer='adam', metrics=['accuracy'])
# fitModel = model.fit(x_train, y_train, epochs=40, batch_size=512, validation_data=(x_val, y_val), verbose=1)
# results = model.evaluate(test_data, test_labels)

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, None, 16)          160000    
                                                                 
 global_average_pooling1d_12  (None, 16)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_24 (Dense)            (None, 16)                272       
                                                                 
 dense_25 (Dense)            (None, 1)                 17        
                                                                 
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________
Epoch 1/40
30/30 [==============================] - 2s 28ms/step - loss: 0.6914 - accuracy: 0.5691 - val_loss: 0.

In [ ]:
model = keras.models.load_model("model.h5")

review_bad = "This is the worst movie I have ever seen Death would have been sweeter than watching this awful awful awful movie"
review_good = "holy shit this movie was epic I could watch this movie every day and still not watch it enough unbelievable screenplay and acting preformances all around"
